In [2]:
import os

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

In [4]:
loader = PyMuPDFLoader("industry_laws.pdf")
pages = loader.load()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# 문서를 문장으로 분리
## 청크 크기 500, 각 청크의 50자씩 겹치도록 청크를 나눈다
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages)

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

/tmp/ipykernel_986/389337336.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# 벡터 저장소 생성
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
## 현재 경로에 'vectorstore' 경로 생성
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")

Vectorstore created and persisted


/tmp/ipykernel_986/157380280.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [14]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조
model = ChatOllama(model="llama3.1", temperature=0)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 10})

In [32]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# Prompt 템플릿 생성
template = '''친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 객관적으로 답하자. 모든 대답은 한국어(Korean)으로 대답해줘.
사고경위를 언급하면 사고상황을 먼저 이해하고 이해한 상황을 나한테 언급해줘, 그리고 산업안전보건기준에 관한 규칙에서 관련된 법령 내용 5가지를 찾아서 나한테 알려줘.
관련된 법령을 개조식으로 나열해줘.":
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [36]:
# Chain 실행
query = "재해자가 오전 TBM(AM07:05)실시후 철골작업을 하기 위해 승강트랩을 오르던중 약 5M높이에서 떨어짐, 여기서 승강트랩이 뭐야?"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

KeyboardInterrupt: 